# Sentinel-1 InSAR Processing

이 노트북은 Sentinel-1 SLC 데이터를 사용하여 DInSAR(Differential Interferometric SAR) 처리를 수행합니다.

## 목표
- SLC 데이터 전처리
- 간섭도(Interferogram) 생성
- 위상 언래핑(Phase Unwrapping)
- 지표 변위 맵 생성

## 필요 리소스 💻

### 최소 사양
- **CPU**: 4 cores (Intel i5 이상)
- **RAM**: 16 GB
- **Disk**: 50 GB 여유 공간
- **처리 시간**: 2-4 시간 (간섭쌍 1개 기준)

### 권장 사양
- **CPU**: 8+ cores (Intel i7/i9, AMD Ryzen 7/9)
- **RAM**: 32 GB 이상
- **Disk**: 100 GB SSD
- **GPU**: CUDA 지원 (SNAP GPU 가속용)
- **처리 시간**: 1-2 시간

### 클라우드 옵션 ☁️
- **AWS EC2**: c5.2xlarge (8 vCPU, 16 GB RAM) - ~$0.34/hr
- **Google Compute**: n2-standard-8 (8 vCPU, 32 GB RAM) - ~$0.39/hr
- **Azure**: Standard_D8s_v3 (8 vCPU, 32 GB RAM) - ~$0.38/hr

## Step 1: Environment Setup

### Windows 환경 체크

**Windows에서 코드로 InSAR 처리하려면 WSL2가 필요합니다!**

#### 옵션 1: WSL2 설치 (권장 ⭐)
```bash
# Windows PowerShell (관리자 권한)
wsl --install -d Ubuntu-22.04

# 재부팅 후 Ubuntu 실행
wsl

# Conda 설치 (WSL 내부)
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh

# InSAR 환경 구성
conda create -n insar python=3.9
conda activate insar
conda install -c conda-forge gdal rasterio geopandas isce2
pip install asf-search jupyter

# Jupyter에서 WSL 커널 사용 가능!
```

#### 옵션 2: Python InSAR 라이브러리 (실험적)
```bash
# Windows에서도 작동 (기능 제한적)
pip install sarpy pyinsar
```

#### 옵션 3: SNAP snappy (복잡함)
- GUI 설치 필수
- Python API 설정 복잡

In [2]:
import sys
import platform
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import zipfile
from datetime import datetime
import xml.etree.ElementTree as ET
from rich.console import Console

console = Console()

# Check operating system
os_name = platform.system()
print(f"Operating System: {os_name}")
if os_name == "Windows":
    print("⚠️  Windows detected! InSAR software requires WSL2 or Linux.")
    print("   See installation guide above.")
elif os_name == "Linux":
    print("✓ Linux detected - InSAR software can be installed")
elif os_name == "Darwin":
    print("✓ macOS detected - InSAR software can be installed")

print("\n✓ Libraries loaded successfully")

# Check available InSAR software
print("\n=== InSAR Software Check ===")
insar_available = False

try:
    import isce
    print("✓ ISCE2 available")
    insar_available = True
except ImportError:
    print("✗ ISCE2 not installed")

try:
    import pygmtsar
    print("✓ PyGMTSAR available")
    insar_available = True
except ImportError:
    print("✗ PyGMTSAR not installed")

try:
    import snappy
    print("✓ SNAP Python API available")
    insar_available = True
except ImportError:
    print("✗ SNAP Python API not installed")

# Alternative Python libraries
try:
    import sarpy
    print("✓ SarPy available (basic SAR processing)")
    insar_available = True
except ImportError:
    print("✗ SarPy not installed")

if not insar_available:
    print("\n⚠️  No InSAR software found!")
    if os_name == "Windows":
        print("\n💡 Recommended: Install WSL2")
        print("   PowerShell (Admin): wsl --install -d Ubuntu-22.04")
        print("   Then install ISCE2 in WSL")
    else:
        print("\n💡 Install InSAR software:")
        print("   conda install -c conda-forge isce2")
else:
    print("\n✓ Ready for InSAR processing!")

Operating System: Windows
⚠️  Windows detected! InSAR software requires WSL2 or Linux.
   See installation guide above.

✓ Libraries loaded successfully

=== InSAR Software Check ===
✗ ISCE2 not installed
✗ PyGMTSAR not installed
✗ SNAP Python API not installed
✗ SarPy not installed

⚠️  No InSAR software found!

💡 Recommended: Install WSL2
   PowerShell (Admin): wsl --install -d Ubuntu-22.04
   Then install ISCE2 in WSL


## Step 2: Load SLC Data

In [ ]:
# Data directory
data_dir = Path('../data/raw')
slc_files = sorted(list(data_dir.glob('*.zip')))

print("=" * 80)
print("SLC Data Files")
print("=" * 80)
print(f"\nData directory: {data_dir}")
print(f"Number of SLC files: {len(slc_files)}\n")

if len(slc_files) >= 2:
    master_file = slc_files[0]
    slave_file = slc_files[1]
    
    print(f"Master: {master_file.name}")
    print(f"  Size: {master_file.stat().st_size / (1024**3):.2f} GB")
    
    print(f"\nSlave: {slave_file.name}")
    print(f"  Size: {slave_file.stat().st_size / (1024**3):.2f} GB")
else:
    print("⚠️  At least 2 SLC files are required for InSAR processing!")
    print("Download data first: python run_data_search.py --download --max-products 2")

## Step 3: InSAR Processing Workflow

### 처리 단계 개요

```
1. Co-registration      → Image alignment (Master-Slave)
2. Interferogram        → Phase difference calculation
3. Topographic Phase    → DEM-based topographic phase removal
4. Filtering            → Noise reduction (Goldstein filter)
5. Phase Unwrapping     → Remove 2π ambiguity
6. Geocoding            → Radar → Geographic coordinates
7. Displacement Map     → Generate displacement map (mm)
```

### Processing Time Estimates ⏱️

| Step | Time (16GB RAM) | Time (32GB RAM) |
|------|----------------|----------------|
| Co-registration | 10-15 min | 5-10 min |
| Interferogram | 5-10 min | 3-5 min |
| Filtering | 10-15 min | 5-10 min |
| Phase Unwrapping | 30-60 min | 15-30 min |
| Geocoding | 10-15 min | 5-10 min |
| **Total** | **~2-4 hours** | **~1-2 hours** |

### Disk Space Requirements 💾

- Input SLC (2 files): ~8 GB
- Intermediate files: ~20 GB
- Final outputs: ~2 GB
- **Total**: ~30 GB

## Step 4: InSAR Processing (SNAP Example)

SNAP은 ESA에서 제공하는 공식 Sentinel-1 처리 도구입니다.

**⚠️ 주의**: 실제 처리는 시간이 오래 걸립니다. 테스트용으로는 작은 영역만 선택하세요.

In [ ]:
# SNAP Processing Example (requires snappy installation)
try:
    import snappy
    from snappy import ProductIO, GPF
    
    print("=" * 80)
    print("SNAP InSAR Processing")
    print("=" * 80)
    
    # Read SLC products
    print("\n1. Reading Master SLC...")
    master = ProductIO.readProduct(str(master_file))
    print(f"   ✓ Master loaded: {master.getName()}")
    
    print("\n2. Reading Slave SLC...")
    slave = ProductIO.readProduct(str(slave_file))
    print(f"   ✓ Slave loaded: {slave.getName()}")
    
    # Processing steps would continue here...
    # (Full processing code omitted for brevity - see SNAP documentation)
    
    print("\n💡 Full SNAP processing workflow:")
    print("   1. TOPSAR-Split")
    print("   2. Apply-Orbit-File")
    print("   3. Back-Geocoding (Co-registration)")
    print("   4. Interferogram Formation")
    print("   5. TOPSAR-Deburst")
    print("   6. TopoPhaseRemoval")
    print("   7. GoldsteinPhaseFiltering")
    print("   8. PhaseUnwrapping (SNAPHU)")
    print("   9. PhaseToDisplacement")
    print("   10. Terrain-Correction (Geocoding)")
    
    print("\n✓ For complete processing code, see SNAP documentation:")
    print("   https://step.esa.int/main/doc/tutorials/")
    
except ImportError:
    print("⚠️  SNAP Python API (snappy) is not installed.")
    print("\nInstallation guide:")
    print("1. Download SNAP: https://step.esa.int/main/download/snap-download/")
    print("2. Configure snappy:")
    print("   cd <SNAP_INSTALL>/bin")
    print("   ./snappy-conf <PYTHON_EXE>")
    print("\nAlternative: Use ISCE2 or PyGMTSAR (see documentation)")
except Exception as e:
    print(f"\n❌ Error: {e}")

## Step 5: Visualize Results

처리가 완료되면 변위 맵을 시각화합니다.

In [ ]:
# Load and visualize displacement map
try:
    import rasterio
    from rasterio.plot import show
    
    # Check if processed data exists
    processed_dir = Path('../data/processed')
    geotiff_files = list(processed_dir.glob('*displacement*.tif'))
    
    if geotiff_files:
        displacement_file = geotiff_files[0]
        
        print(f"Loading displacement map: {displacement_file.name}")
        
        with rasterio.open(displacement_file) as src:
            displacement = src.read(1)
            
            # Plot
            fig, ax = plt.subplots(figsize=(12, 10))
            
            im = ax.imshow(displacement, cmap='RdYlBu_r', vmin=-50, vmax=50)
            ax.set_title('Ground Displacement Map (mm)', fontsize=14, fontweight='bold')
            ax.set_xlabel('Longitude', fontsize=12)
            ax.set_ylabel('Latitude', fontsize=12)
            
            # Colorbar
            cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            cbar.set_label('Displacement (mm)', rotation=270, labelpad=20, fontsize=12)
            
            plt.tight_layout()
            plt.show()
            
            # Statistics
            print("\n=== Displacement Statistics ===")
            print(f"Min: {np.nanmin(displacement):.2f} mm")
            print(f"Max: {np.nanmax(displacement):.2f} mm")
            print(f"Mean: {np.nanmean(displacement):.2f} mm")
            print(f"Std: {np.nanstd(displacement):.2f} mm")
    else:
        print("⚠️  No processed displacement map found.")
        print("Run InSAR processing first (Step 4)")
        
except ImportError:
    print("⚠️  rasterio is required for visualization")
    print("Install: pip install rasterio")
except Exception as e:
    print(f"Error: {e}")

## Next Steps 🚀

### Completed ✅
- InSAR processing workflow understanding
- Software requirements check
- Resource estimation

### To Do 📝

1. **Install InSAR Software**
   - SNAP (recommended for beginners)
   - ISCE2 (for advanced users)
   - PyGMTSAR (for automation)

2. **Run Processing**
   - Use SNAP GUI for first-time processing
   - Automate with Python after understanding workflow

3. **Time-series Analysis** → `03_time_series_analysis.ipynb`
   - SBAS (Small Baseline Subset)
   - Velocity estimation
   - Atmospheric correction

4. **Advanced Visualization** → `04_visualization.ipynb`
   - Interactive maps (Folium)
   - 3D visualization

---

### Installation Guides 📚

#### SNAP Installation
```bash
# 1. Download SNAP from:
#    https://step.esa.int/main/download/snap-download/

# 2. Install SNAP (GUI installer)

# 3. Configure Python API:
cd <SNAP_INSTALL_DIR>/bin
./snappy-conf <YOUR_PYTHON_EXE>

# 4. Test installation:
python -c "import snappy; print(snappy.__version__)"
```

#### ISCE2 Installation
```bash
# Using conda (recommended):
conda install -c conda-forge isce2

# Test:
python -c "import isce; print(isce.__version__)"
```

#### PyGMTSAR Installation
```bash
pip install pygmtsar

# Test:
python -c "import pygmtsar; print(pygmtsar.__version__)"
```

---

**💡 Tip**: Start with SNAP GUI to understand the workflow, then automate with Python scripts.